<a href="https://colab.research.google.com/github/imranajec/Synthetic-Data-Generation/blob/main/FeedbackSegragation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
import pandas as pd

# Create the dataset
data = {
    "feedback": [
        "The waiting time for appointments is too long.",
        "The hospital cafeteria food quality needs improvement.",
        "Dr. Smith was excellent, very thorough and patient.",
        "The nurses were very kind and helpful during my stay.",
        "The billing process is confusing and took too long.",
        "The hospital rooms are not as clean as expected.",
        "The receptionist was rude and unhelpful.",
        "The parking facilities are inadequate and always full.",
        "The overall service at the hospital was unsatisfactory.",
        "The medical equipment seems outdated and not well-maintained."
        "rohit did not do my surgery properly"
    ]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
csv_file_path = 'patient_feedback.csv'
df.to_csv(csv_file_path, index=False)

print(f"Data saved to {csv_file_path}")


Data saved to patient_feedback.csv


In [2]:

!pip install langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 355.8/355.8 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.4
    Uninstalling google-ai-generativelanguage-0.6.4:
      Successfully uninstalled google-ai-generativelanguage-0.6.4
  Attempting uninstall: google-generativeai
    Found existing installation: google-generativeai 0.5.4
    Uninstalling google-generativeai-0.5.4:
      Successfully uninstalled google-generativeai-0.5.4


In [68]:
import os
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

In [69]:
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [74]:
import pandas as pd
import json
from langchain_google_genai import ChatGoogleGenerativeAI

# Function to process feedback with Gemini Pro
def process_feedback_with_gemini(feedback):
    prompt = (
        f"Feedback: {feedback}\n"
        "Please provide the following information in JSON format:\n"
        "1. Sentiment (Positive/Negative)\n"
        "2. Is the problem valid (yes/no)?\n"
        "3. If valid, describe the problem.\n"
        "4. Who cause the problem, answer in one or two words?\n If a person name is mentioned, genralize him/her to his job role. \n"
        "5. Specific entity name (if any).\n"
    )
    response = llm.invoke(prompt)
    return response

# Read the feedback dataset
df = pd.read_csv('/content/patient_feedback.csv')

# Process each feedback entry
processed_feedback = []
for index, row in df.iterrows():
    feedback = row['feedback']
    response = process_feedback_with_gemini(feedback)
    response_content = response.content
    processed_feedback.append(response_content)

# Convert the processed feedback to JSON
processed_json = json.dumps(processed_feedback, indent=2)

# Print the JSON
print(processed_json)


[
  "```json\n{\n  \"Sentiment\": \"Negative\",\n  \"Is the problem valid\": \"yes\",\n  \"Describe the problem\": \"The waiting time for appointments is too long.\",\n  \"Who cause the problem\": \"Doctor\",\n  \"Specific entity name\": null\n}\n```",
  "```JSON\n{\n  \"Sentiment\": \"Negative\",\n  \"Is the problem valid\": \"yes\",\n  \"Describe the problem\": \"The food quality in the hospital cafeteria needs improvement.\",\n  \"Who cause the problem\": \"Cafeteria staff\",\n  \"Specific entity name\": null\n}\n```",
  "```\n{\n  \"Sentiment\": \"Positive\",\n  \"Is_problem_valid\": \"No\",\n  \"Problem_description\": \"N/A\",\n  \"Who_caused_the_problem\": \"N/A\",\n  \"Specific_entity_name\": \"N/A\"\n}\n```",
  "```\n{\n  \"Sentiment\": \"Positive\",\n  \"Is_problem_valid\": \"no\",\n  \"Problem_description\": \"N/A\",\n  \"Problem_cause\": \"N/A\",\n  \"Specific_entity\": \"N/A\"\n}\n```",
  "```json\n{\n  \"Sentiment\": \"Negative\",\n  \"Is_problem_valid\": \"yes\",\n  \"Pro

In [75]:
import re

# Function to clean and parse the JSON responses
def clean_and_parse_json(response):
    # Remove the code block delimiters and strip whitespace
    clean_response = re.sub(r'^```[a-zA-Z]*\n|\n```$', '', response).strip()
    try:
        # Parse the JSON response
        parsed_json = json.loads(clean_response)
        return json.dumps(parsed_json, indent=4)
    except json.JSONDecodeError as e:
        # Handle JSON parsing errors
        return f"Invalid JSON response: {clean_response}\nError: {e}"

# Initialize an empty list to hold cleaned responses
cleaned_responses = []

# Pretty-print each JSON response
for response in processed_feedback:
    if response:  # Check if the response is not empty
        cleaned_responses.append(clean_and_parse_json(response))
    else:
        cleaned_responses.append("Empty response")

# Print each cleaned response with proper separation
for cleaned_response in cleaned_responses:
    print(cleaned_response)
    print("\n" + "-"*50 + "\n")  # Separator for readability


{
    "Sentiment": "Negative",
    "Is the problem valid": "yes",
    "Describe the problem": "The waiting time for appointments is too long.",
    "Who cause the problem": "Doctor",
    "Specific entity name": null
}

--------------------------------------------------

{
    "Sentiment": "Negative",
    "Is the problem valid": "yes",
    "Describe the problem": "The food quality in the hospital cafeteria needs improvement.",
    "Who cause the problem": "Cafeteria staff",
    "Specific entity name": null
}

--------------------------------------------------

{
    "Sentiment": "Positive",
    "Is_problem_valid": "No",
    "Problem_description": "N/A",
    "Who_caused_the_problem": "N/A",
    "Specific_entity_name": "N/A"
}

--------------------------------------------------

{
    "Sentiment": "Positive",
    "Is_problem_valid": "no",
    "Problem_description": "N/A",
    "Problem_cause": "N/A",
    "Specific_entity": "N/A"
}

--------------------------------------------------

{
    

In [64]:
import re
import pandas as pd
import json
from langchain_google_genai import ChatGoogleGenerativeAI

# Function to process feedback with Gemini Pro
def process_feedback_with_gemini(feedback_list):
    prompt = "Please provide the following information in JSON format for each feedback:\n"

    for i, feedback in enumerate(feedback_list):
        prompt += (
            f"**Feedback {i+1}:** {feedback}\n"
            "1. Sentiment (Positive/Negative)\n"
            "2. Is the problem valid (yes/no)?\n"
            "3. If valid, describe the problem.\n"
            "4. Who caused the problem? Answer in one or two words. If there is a name of the person, entity, or object, try to identify their profession or the department or group they belong to based on the meaning of the sentence.\n"
            "5. Specific entity name (if any).\n\n"
        )

    response = llm.invoke(prompt)
    return response

# Read the feedback dataset
df = pd.read_csv('/content/patient_feedback.csv')

# Extract all feedback entries
feedback_list = df['feedback'].tolist()

# Process all feedback entries in a single LLM invocation
response = process_feedback_with_gemini(feedback_list)

# Assuming the response content is structured in a way that it can be split back into individual responses
response_content = response.content.strip().split('\n\n')

# Initialize a list to hold the cleaned JSON responses
cleaned_responses = []

# Remove code block delimiters and parse each response
for content in response_content:
    # Remove Markdown code block delimiters
    clean_content = re.sub(r'```json\n|```JSON\n|```\n|```', '', content).strip()
    try:
        parsed_json = json.loads(clean_content)
        cleaned_responses.append(parsed_json)
    except json.JSONDecodeError as e:
        cleaned_responses.append(f"Invalid JSON response: {clean_content}\nError: {e}")

# Convert the processed feedback to JSON
processed_json = json.dumps(cleaned_responses, indent=2)

# Print the JSON
print(processed_json)


[
  "Invalid JSON response: **Feedback 1:**\n{\n  \"Sentiment\": \"Negative\",\n  \"Is the problem valid\": \"yes\",\n  \"Problem description\": \"Waiting time for appointments is too long.\",\n  \"Who caused the problem\": \"Hospital\",\n  \"Specific entity name\": null\n}\nError: Expecting value: line 1 column 1 (char 0)",
  "Invalid JSON response: **Feedback 2:**\n{\n  \"Sentiment\": \"Negative\",\n  \"Is the problem valid\": \"yes\",\n  \"Problem description\": \"Hospital cafeteria food quality needs improvement.\",\n  \"Who caused the problem\": \"Cafeteria staff\",\n  \"Specific entity name\": null\n}\nError: Expecting value: line 1 column 1 (char 0)",
  "Invalid JSON response: **Feedback 3:**\n{\n  \"Sentiment\": \"Positive\",\n  \"Is the problem valid\": \"no\",\n  \"Problem description\": \"Dr. Smith provided excellent service.\",\n  \"Who caused the problem\": null,\n  \"Specific entity name\": \"Dr. Smith\"\n}\nError: Expecting value: line 1 column 1 (char 0)",
  "Invalid JS

In [23]:
print(response)

content='```\n{\n  "Sentiment": "Negative",\n  "Is the problem valid": "yes",\n  "Describe the problem": "The medical equipment seems outdated and not well-maintained.",\n  "Who cause the problem": "Hospital",\n  "Specific entity name": "N/A"\n}\n```' response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-e7af6480-4a3e-4ec2-bcc9-0e0f1981f1bb-0' usage_metadata={'input_tokens': 81, 'output_tokens': 68, 'total_tokens': 149}


In [44]:
import pandas as pd
import json
from langchain_google_genai import ChatGoogleGenerativeAI

# Function to process feedback with Gemini Pro
def process_feedback_with_gemini(feedback_list):
    prompt = "Please provide the following information in JSON format for each feedback:\n"

    for i, feedback in enumerate(feedback_list):
        prompt += (
            f"Feedback {i+1}: {feedback}\n"
            "1. Sentiment (Positive/Negative)\n"
            "2. Is the problem valid (yes/no)?\n"
            "3. If valid, describe the problem.\n"
            "4. Who caused the problem? Answer in one or two words. If there is a name of the person, entity, or object, try to identify their profession or the department or group they belong to based on the meaning of the sentence.\n"
            "5. Specific entity name (if any).\n\n"
        )

    response = llm.invoke(prompt)
    return response

# Read the feedback dataset
df = pd.read_csv('/content/patient_feedback.csv')

# Extract all feedback entries
feedback_list = df['feedback'].tolist()

# Process all feedback entries in a single LLM invocation
response = process_feedback_with_gemini(feedback_list)

# Assuming the response content is structured in a way that it can be split back into individual responses
response_content = response.content.strip().split('\n\n')  # Adjust the splitting logic based on actual response structure

# Ensure the number of responses matches the number of feedback entries
if len(response_content) != len(feedback_list):
    raise ValueError("The number of responses from the LLM does not match the number of feedback entries.")

# Create a list to hold processed feedback
processed_feedback = []

# Process each individual response
for content in response_content:
    try:
        parsed_json = json.loads(content)
        processed_feedback.append(parsed_json)
    except json.JSONDecodeError as e:
        processed_feedback.append(f"Invalid JSON response: {content}\nError: {e}")

# Convert the processed feedback to JSON
processed_json = json.dumps(processed_feedback, indent=2)

# Print the JSON
print(processed_json)


[
  "Invalid JSON response: **Feedback 1:**\n```json\n{\n  \"Sentiment\": \"Negative\",\n  \"Is valid\": \"yes\",\n  \"Problem description\": \"The waiting time for appointments is too long.\",\n  \"Who caused the problem\": \"Hospital\",\n  \"Specific entity name\": \"N/A\"\n}\n```\nError: Expecting value: line 1 column 1 (char 0)",
  "Invalid JSON response: **Feedback 2:**\n```json\n{\n  \"Sentiment\": \"Negative\",\n  \"Is valid\": \"yes\",\n  \"Problem description\": \"The hospital cafeteria food quality needs improvement.\",\n  \"Who caused the problem\": \"Cafeteria staff\",\n  \"Specific entity name\": \"N/A\"\n}\n```\nError: Expecting value: line 1 column 1 (char 0)",
  "Invalid JSON response: **Feedback 3:**\n```json\n{\n  \"Sentiment\": \"Positive\",\n  \"Is valid\": \"yes\",\n  \"Problem description\": \"Dr. Smith provided excellent service.\",\n  \"Who caused the problem\": \"Dr. Smith\",\n  \"Specific entity name\": \"Dr. Smith\"\n}\n```\nError: Expecting value: line 1 col

In [13]:
processed_df.to_csv('processed_feedback.csv', index=False)

print("Feedback processing complete. Results saved to 'processed_feedback.csv'.")


Feedback processing complete. Results saved to 'processed_feedback.csv'.


In [20]:
import pandas as pd
import json
from langchain_google_genai import ChatGoogleGenerativeAI


def process_feedback_with_gemini(feedback):
    prompt = (
        f"Feedback: {feedback}\n"
        "Please provide the following information in JSON format:\n"
        "1. Sentiment (Positive/Negative)\n"
        "2. Is the problem valid (yes/no)?\n"
        "3. If valid, describe the problem.\n"
        "4. Who is the problem for (Doctor, Receptionist, Room, Nurse, Hospital)?\n"
        "5. Specific entity name (if any).\n"
        "6. Relevant code of conduct or rules."
    )
    response = llm.invoke(prompt)
    print(response)  # Inspect the response object
    response_json = extract_json_from_response(response.content)
    return response_json

# Function to extract JSON from the response content
def extract_json_from_response(response_content):
    try:
        start_index = response_content.index('{')
        end_index = response_content.rindex('}') + 1
        json_content = response_content[start_index:end_index]
        return json.loads(json_content)
    except (ValueError, json.JSONDecodeError) as e:
        print(f"Error parsing JSON from response: {e}")
        return {}

# Read the feedback dataset
df = pd.read_csv('/content/patient_feedback.csv')

# Process each feedback entry
processed_feedback = []
for index, row in df.iterrows():
    feedback = row['feedback']
    response_json = process_feedback_with_gemini(feedback)
    processed_feedback.append(response_json)

# Convert the processed feedback to a DataFrame
processed_df = pd.DataFrame(processed_feedback)

# Save the processed feedback to a new CSV file
processed_df.to_csv('processed_feedback.csv', index=False)

print("Feedback processing complete. Results saved to 'processed_feedback.csv'.")
:




content='```\n{\n  "Sentiment": "Negative",\n  "Is_the_problem_valid": "yes",\n  "Problem_description": "The waiting time for appointments is too long.",\n  "Problem_for": "Hospital",\n  "Specific_entity_name": "N/A",\n  "Relevant_code_of_conduct_or_rules": "N/A"\n}\n```' response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-9d85e0de-fa41-4942-8dac-f735279bfe4b-0' usage_metadata={'input_tokens': 93, 'output_tokens': 90, 'total_tokens': 183}
content='```json\n{\n  "Sentiment": "Negative",\n  "Is the problem valid": "yes",\n  "Describe the probl

In [21]:
processed_df.to_csv('processed_feedback.csv', index=False)

print("Feedback processing complete. Results saved to 'processed_feedback.csv'.")

Feedback processing complete. Results saved to 'processed_feedback.csv'.
